In [166]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [167]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "AVG-AE"

In [168]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [169]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,-0.068412,-0.026547,0.588228,0.469505,0.805360,-0.235864,-0.814815,-0.793692,-0.326378,...,-0.530038,-0.884693,0.955239,1.061715,1.291020,-0.616202,-0.003106,0.125670,-0.623129,-0.721230
1,CHUM-002,-0.635281,-0.318902,1.857928,0.742818,1.243504,1.599360,-0.595187,0.223939,0.970232,...,0.029161,-0.145592,0.983818,-0.878192,0.683243,-3.209786,1.971604,-0.341882,-0.070023,-1.282687
2,CHUM-006,-1.269402,0.375295,0.906491,0.669751,-0.295800,-0.593660,0.287808,-0.900831,-0.232552,...,-0.263073,1.799599,-0.584013,-1.037655,-3.352272,-2.464304,-1.141363,-0.128082,-1.781053,-1.429000
3,CHUM-007,-1.371328,-0.284073,0.652182,0.842345,0.991431,0.230390,-0.815961,0.355382,1.500634,...,-0.057167,0.156637,0.823456,1.074826,-0.005615,-0.710393,-0.189970,-0.315353,-0.393455,-0.102896
4,CHUM-008,-0.796137,0.299490,0.393766,0.842857,0.833519,-0.088249,-1.937676,0.311921,2.518222,...,0.379776,0.656207,0.638874,-0.100001,0.059302,-2.408928,1.095648,-0.142162,-2.151676,0.583017


# Up to here

Read Outcome csv

In [170]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [171]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [172]:
OD.columns[0]

'Patient_ID'

In [173]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [174]:
OD.columns[0]


'Patient_ID'

In [175]:
PET_outcome_data.columns[0]

'Patient_ID'

In [176]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,-0.068412,-0.026547,0.588228,0.469505,0.805360,-0.235864,-0.814815,-0.793692,-0.326378,...,-0.530038,-0.884693,0.955239,1.061715,1.291020,-0.616202,-0.003106,0.125670,-0.623129,-0.721230
1,CHUM-002,-0.635281,-0.318902,1.857928,0.742818,1.243504,1.599360,-0.595187,0.223939,0.970232,...,0.029161,-0.145592,0.983818,-0.878192,0.683243,-3.209786,1.971604,-0.341882,-0.070023,-1.282687
2,CHUM-006,-1.269402,0.375295,0.906491,0.669751,-0.295800,-0.593660,0.287808,-0.900831,-0.232552,...,-0.263073,1.799599,-0.584013,-1.037655,-3.352272,-2.464304,-1.141363,-0.128082,-1.781053,-1.429000
3,CHUM-007,-1.371328,-0.284073,0.652182,0.842345,0.991431,0.230390,-0.815961,0.355382,1.500634,...,-0.057167,0.156637,0.823456,1.074826,-0.005615,-0.710393,-0.189970,-0.315353,-0.393455,-0.102896
4,CHUM-008,-0.796137,0.299490,0.393766,0.842857,0.833519,-0.088249,-1.937676,0.311921,2.518222,...,0.379776,0.656207,0.638874,-0.100001,0.059302,-2.408928,1.095648,-0.142162,-2.151676,0.583017


In [177]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [178]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [179]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,-0.068412,-0.026547,0.588228,0.469505,0.805360,-0.235864,-0.814815,-0.793692,-0.326378,...,0.955239,1.061715,1.291020,-0.616202,-0.003106,0.125670,-0.623129,-0.721230,0,1704
1,CHUM-002,-0.635281,-0.318902,1.857928,0.742818,1.243504,1.599360,-0.595187,0.223939,0.970232,...,0.983818,-0.878192,0.683243,-3.209786,1.971604,-0.341882,-0.070023,-1.282687,1,439
2,CHUM-006,-1.269402,0.375295,0.906491,0.669751,-0.295800,-0.593660,0.287808,-0.900831,-0.232552,...,-0.584013,-1.037655,-3.352272,-2.464304,-1.141363,-0.128082,-1.781053,-1.429000,0,1186
3,CHUM-007,-1.371328,-0.284073,0.652182,0.842345,0.991431,0.230390,-0.815961,0.355382,1.500634,...,0.823456,1.074826,-0.005615,-0.710393,-0.189970,-0.315353,-0.393455,-0.102896,0,1702
4,CHUM-008,-0.796137,0.299490,0.393766,0.842857,0.833519,-0.088249,-1.937676,0.311921,2.518222,...,0.638874,-0.100001,0.059302,-2.408928,1.095648,-0.142162,-2.151676,0.583017,0,1499


In [180]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)